## Practical Exercise 1: word2vec
By Joline Janz and Frederik Wollatz

Each Notebook will contribute equaly to your final grade.

This practical Exercise is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. 

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In this Notebook you will learn the basics on how to construct a word-embedding. As you recall from the lecture, word-embeddings are a type of word representation that allows words with similar meaning to have a similar representation. To do this, words are represented as real-valued vectors in a predefined vector space. Additionally, you will also learn how to use some basic NLP tools like tokenization and regular Expressions!
Good Luck!

In [2]:
import numpy as np
import os
from random import shuffle
import re

In [3]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
try: 
    import nltk
except:
    import sys #Here we install nltk. You only have to execute this cell once!
    !{sys.executable} -m pip install nltk 
    import nltk
    nltk.download()

### Part 0: Download the TED dataset

As input we need a large amount of text data. We will use the TED database, which are the transcripts of Ted Talks. The next cells will download everything you need, this might take a while as the dataset is 75MB.

In [5]:
import urllib.request
import zipfile
import lxml.etree

In [6]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [7]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

Before using our text, we need to preprocess it. Therefore, we bring it into a form that is predictable and analyzable. We attempt to clean up the raw subtitles a bit, so that we get only complete sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

<h4>Execercise 1.1 (2 Points)</h4> 
Before we work with the data we should have a look at it. We already marked some areas for you, that need to be cleaned. You do not have to code anything here, you just have to become aware of sensitive preprocessing steps.

In [8]:
#Have a look at the output of this code, to see some examples
i = input_text.find("Hyowon Gweon: See this?")
print(input_text[i:i+145])


i = input_text.find("You will earn")
print(input_text[i:i+30])

Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.
Laura Schulz: Told you. (Laughs)

You will earn 10% of any gold 


For example the parenthesized strings like "(Ball squeaks)" and symbols like % could distort the semantics of words in the embedding. Name at least two more problematic sections and how you would solve them.


<b>Your Solution:</b> 
<br>- Parenthesized Strings
<br>- Percent-Symbol
<br>- Speaker Identifier (e.g. "Hyowon Gweon:")
<br>- Number-Symbol


<h4>Exercise 1.2 (2 Points)</h4>
Let's start by removing all parenthesized strings using a regex:

In [9]:
i = input_text.find("Hyowon Gweon: See this?")
print("before")
print(input_text[i:i+93])

input_text_noparens = re.sub(r'\([^)]*\)', '', input_text) #Identifies everything in parenthesis and replaces it with ""


#you can use this to verify
i = input_text_noparens.find("Hyowon Gweon: See this?")
print("after")
print(input_text_noparens[i:i+93])

#We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

before
Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (
after
Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.
Laura Schulz: Told you.


Try it yourself: Replace every percentage Symbol with the word "percent"

In [12]:
i = input_text_noparens.find("You will earn")
print("before")
print(input_text_noparens[i:i+30])

input_text_clean = re.sub("%", " percent", input_text_noparens)

i = input_text_clean.find("You will earn")
print("after")
print(input_text_clean[i:i+37])

before
You will earn 10% of any gold 
after
You will earn 10 percent of any gold 


<h4>Exercise 1.3 (up to 6 Points)</h4>
Now you have learned how to use RegEx to your advantage and have Identified potential parts of the text, that we want to eliminate. We have already implented how to remove all parenthesized strings. It is now your Task to implement at least one more heuristic to replace the problematic parts you previously identified(Exercise 1.1).
A simple replacing function (like the one where you replaced "%" with "percent") gives you two points per function, more complicated implementations will give you the full 6 Points.

In [ ]:
from num2words import num2words

search_str = "by does."

i = input_text_clean.find(search_str) #find problematic parts
print("before")
print(input_text_clean[i:i+37]) #and show them

# Remove speaker identifiers. Unlike in 1.4, we only remove
# substrings before ':' that either consist of two capital letters
# for initials or is two words starting with capital letters.
# Thus, the problem from 1.4 is resolved, though this ofc
# also isn't a perfect solution (e.g. the speaker identifier "Ray:"
# does not get detected with this method.
new_input_text_clean = re.sub(
    r'([A-Z]{2}|[A-Z][a-z]+\s[A-Z][a-z]+):', '', input_text_clean)

i = new_input_text_clean.find(search_str) #validate your method
print("after")
print(new_input_text_clean[i:i+37])


search_str = "You will earn"
i = new_input_text_clean.find(search_str)
print("before")
print(new_input_text_clean[i:i+30])


def num_to_word(line):
    """
    finds and replaces FIRST number in numeric format with
    word representation.

    :param line:
    :return: if digit to replace was found, slightly cleaned line
    """

    # check in line for first number occurence
    m = re.search(r'(?P<number>[0-9]+)?(?P<postnumber>.*)', line)
    num_to_replace = m.groupdict()['number']
    # if number in numeric format exists, replace
    if num_to_replace:
        line = re.sub(
            num_to_replace,
            num2words(num_to_replace),
            new_input_text_clean
        )
    return bool(num_to_replace), line

X = []
for line in new_input_text_clean.split('\n'):

    match_found, line = num_to_word(line)
    # while there is a match, check for other numbers to replace
    while match_found:
        match_found, line = num_to_word(line)

    # store in X
    X.append(line)

new_input_text_clean="".join(X)

"""
numbers = re.findall(, new_input_text_clean)
for number in numbers:
    new_input_text_clean = re.sub(number, num2words(number), new_input_text_clean)
"""

search_str = "You will earn"
i = new_input_text_clean.find(search_str)
print("after")
print(new_input_text_clean[i:i+30])

before
by does.
 Hyowon Gweon: See this?  Di
after
by does.
  See this?  Did you see tha
before
You will earn 10 percent of an


<h4>Exercise 1.4 (4 Points)</h4>
What does this block of code do? Identify one flaw. Again, you dont have to code anything here!


In [ ]:
i = input_text_clean.find("Hyowon Gweon: See this?")
print("before:")
print(input_text_clean[i-31:i+92])

X = []
for line in input_text_clean.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    X.extend(m.groupdict()['postcolon'])
input_text_clean2="".join(X)


i = input_text_clean2.find("See this?")
print("after:")
print(input_text_clean2[i-31:i+55])

This code block removes the speaker identifiers (e.g. Laura Schulz).
The flaw is that it removes anything at the start of a new line
before the first ":". However, this also detects and thus deletes
strings like 'Second one:' which appear at the start, but are no speaker identifiers
(These can be found by running re.findall(r'\n[^:]{,20}:', new_input_text_clean)).

<h4>Exercise 1.5 (6 Points)</h4>

To build our embedding we need to tokenize every single word. Therefore we first need to split the text into sentences and after that into words. 
Try it yourself or use the NLTK-Tools build for this (https://www.kite.com/python/docs/nltk.word_tokenize + https://www.kite.com/python/docs/nltk.sent_tokenize).
To make it easier to build our Embedding we should also delete every character that is not a letter. Additionally, we could lower vocabulary count. A way to do this is by converting capital characters to lower case characters.

Split your text into sentences and save them in the array `sentences_strings_ted`.
Save one variabale `tokens` with all the tokens in the text and one array named `sentences_ted` that contains an array for every sentence, with all the tokenized words of that sentence.<br><br>
Example:<br>
If the Text looks like this: "I love cake. You have to be honest, you love it too!", the variables look like:<br><br>
sentences_strings_ted=['I love cake.', 'You have to be honest, you love it too!']<br>
sentences_ted=[['i', 'love', 'cake'], ['you', 'have', 'to', 'be', 'honest', 'you', 'love', 'it', 'too']]<br>
tokens=['i', 'love', 'cake', 'you', 'have', 'to', 'be', 'honest', 'you', 'love', 'it', 'too']<br>


Apply this to `new_input_text_clean`

In [ ]:
from nltk.tokenize import RegexpTokenizer

sentences_strings_ted = nltk.sent_tokenize(
    new_input_text_clean
)

# only keeps tokens that consist of lowercase alphabet letters only
reg_tok = RegexpTokenizer(r'[a-z]+')

sentences_ted = \
    [reg_tok.tokenize(sent.lower())
     for sent in sentences_strings_ted]


tokens = \
    [token for sentence in sentences_ted for token in sentence]

# print results
print(sentences_strings_ted[:2])
print(sentences_ted[:2])
print(tokens[:50])

<h4>Exercise 1.6 (1 Point)</h4>
The good side is, that by converting all capital letters is, we reduce the volume of the vocabulary. Thereby we dont differentiate between the the words "today" and "Today". 
Can you think of any downside to this process?

Proper nouns are characterized by their capital letters. If e.g. we would have an entity called 'Blue', but also the color 'blue' in our text, we wouldn't be abe to distinguish between them anymore even though they are different words.

Now we can have a look at the processed dataset:

In [ ]:
len(sentences_ted)

In [ ]:
print(sentences_ted[0])
print(sentences_ted[-1])

### Part 2: Word Frequencies

<h4>Exercise 2.1 (4 Points)</h4>
Your next task will be to store the counts of the top 1000 most frequent words in a list called `counts_ted_top1000` ! There are multiple ways to do this. You can have a look at the Counter-Function(https://docs.python.org/2/library/collections.html) or the FreqDist-Function (https://www.kite.com/python/docs/nltk.FreqDist). If you dont trust any of those you can of course build your own function.
In the end we want an array with tupels of the structure [(WordA,FrequencyA),(WordB,FrequencyB)].

In [ ]:
counts_ted_top1000 = nltk.FreqDist(tokens)

counts_ted_top1000tupels = [(word, counts_ted_top1000[word]) for word in counts_ted_top1000]

The following code is going to plot a histogramm of the distribution of the  top-30 words.

In [ ]:
import matplotlib.pyplot as plt
mostfreqn=30 #Here we define how many of them we want to see in the diagramm 
frequency=[y for (x,y) in counts_ted_top1000tupels][:mostfreqn]
word=[x for (x,y) in counts_ted_top1000tupels][:mostfreqn]
indices = np.arange(len(counts_ted_top1000tupels[:mostfreqn]))
plt.bar(indices, frequency, color='r')
plt.xticks(indices, word, rotation='vertical')
plt.tight_layout()
plt.show()

You can clearly see, that many of the most common words are so called stop words. Stop Words are words, that are tipically not usefull to identify what a text is about.

### Part 3: Train Word2Vec

Now it is time to train the modell. Gensim has an already implemented model that you can use.

Using the provided modell is enough for the purposes of our notebook. If you want to dive deeper into the topic this youtube video https://www.youtube.com/watch?v=kKDYtZfriI8 could be a great guidance for you to get started. 


In [ ]:
#This takes a moment...dont worry :D
from gensim.models import Word2Vec
model_ted = Word2Vec(sentences_ted)

word_vectors = model_ted.wv

### Part 4: Ted Learnt Representations (3 Points)


Finding similar words: (see gensim docs for functions, that might help you https://radimrehurek.com/gensim/models/keyedvectors.html)

Now lets explore what we can do with this! How does "house" look in our embedding?

In [ ]:
word_vectors['house']

What is the most similar word for "town"?

In [ ]:
word_vectors.most_similar(positive="town", topn=1)

How similar are the words "town" and "house"?

In [ ]:
word_vectors.cosine_similarities(word_vectors['house'], [word_vectors['town']])

<h4>Exercise 4.1 (3 Points)</h4>
Now that we have trained our own embedding, lets test some classical ideas: 
implement the following formula. Print out the 10 words, that are most similar to this formula: <br>
$King-Man+Woman=???$
There are two ways of computing similarity in word Embeddings:
 - https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.most_similar.html
 - https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.most_similar_cosmul.html
You should try out both! In this case one of them is better, but both of them are valid methods for computing similarity in the word-space.


In [ ]:
print(word_vectors.most_similar(
    positive=["king", "woman"], negative=["man"]))
print(word_vectors.most_similar_cosmul(
    positive=["king", "woman"], negative=["man"]))

The expected outcome (Queen) should be one of the top ten most similar words. But there are also a lot of words, that you would not expect. Think about where how these words might be connected to the formula. You do not have to write anything down for this task, just take your time and understand why some of the words (luther, mary, dr, president) might be in this list.

#### t-SNE visualization

We will use the t-SNE algorithm, given belwo, for visualization. The so-called t-Distributed Stochastic Neighbor Embedding (t-SNE) is an unsupervised and non-linear machine learning technique. It is commonly used for visualizing high dimensional data (just like our high dimensional vectors). You do not have to understand the code, it's purpose is simply to give you an idea of how the data is arranged in high dimensional space.

<h4>Exercise 4.2 (2 Points)</h4>
To use the t-SNE code below, first put a list of the top 100 words (as strings) into a variable `words_top_ted`. 

In [ ]:
words_top_ted = list(counts_ted_top1000)[:100]

The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [ ]:
# This assumes words_top_ted is a list of strings, the top 250 words
words_top_vec_ted = model_ted[words_top_ted]

The next few lines are for the t-SNE visualization.

In [ ]:
from sklearn.manifold import TSNE 
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)